In [ ]:
import os 
from automaticTB.tools import read_yaml, write_yaml
from automaticTB.solve.interaction import (
    get_InteractingAOSubspaces_from_cluster, CombinedAOSubspaceInteraction
)
from automaticTB.examples import get_cif_filename_for
from automaticTB.solve import (
    get_structure_from_cif_file,
)
from automaticTB.interface import OrbitalPropertyRelationship

bulksi = get_structure_from_cif_file(
    get_cif_filename_for("Si"), {"Si":"3s3p4s"}, rcut = 3.9
)
temporary_folder = "temporary"
all_iao = []
for i, centered_cluster in enumerate(bulksi.centered_clusters):
    temporary_file = os.path.join(temporary_folder, f"2nn_tmp_{i+1}.yaml")
    if os.path.exists(temporary_file):
        tmp_iao = read_yaml(temporary_file)
    else:
        tmp_iao = []
        for ceq_cluster in centered_cluster.centered_equivalent_clusters:
            ceq_cluster.set_symmetry(find_additional_symmetry = True)
            tmp_iao += get_InteractingAOSubspaces_from_cluster(ceq_cluster)
        write_yaml(tmp_iao, temporary_file)
    all_iao += tmp_iao
    
combined = CombinedAOSubspaceInteraction(all_iao)

relationship = OrbitalPropertyRelationship.from_structure_combinedInteraction(bulksi, combined)
relationship.print_free_pairs()

relationship.to_file("Si_2nn_relationship.dat")